In [1]:
from keras.models import Model
from keras.layers import Input, Dense, Reshape, merge
from keras.layers.embeddings import Embedding
from keras.preprocessing.sequence import skipgrams
from keras.preprocessing import sequence
import urllib
import collections
import os
import zipfile
import numpy as np
import tensorflow as tf
import numpy as np
import tensorflow as tf
import helpers
tf.reset_default_graph()
sess = tf.InteractiveSession()
import sys
import json
import re
import collections
import os
import datetime
import helpers

/home/rgsl888/ana/envs/tf27/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Provide the playlist file to create the embeddings

In [3]:
fullpath = 'mpd.slice.1000-1999.json'

In [4]:
# Create the dataset for the embeddings
f = open(fullpath)
file_content = f.read()
f.close()
mpd_slice = json.loads(file_content)

# Input to seq to seq model
X_train = []

# Playlist name list
play_list_names = [re.sub('[^a-zA-Z0-9]+', '', mpd_slice['playlists'][idx]['name'].lower()) for idx in range(0, len(mpd_slice['playlists']))]

# Dictionary to maintain the song index and track name, other features can be added later
track_dict = {}
count  = len(set(play_list_names))
count += 2

# Looping to create the X_train
for idx in range(0, len(mpd_slice['playlists'])):
    lst_current = [] 
    
    lst_current.append(play_list_names.index(re.sub('[^a-zA-Z0-9]+', '', mpd_slice['playlists'][idx]['name'].lower())) + 2)    

    for idy in range(0, len(mpd_slice['playlists'][idx]['tracks'])):
        track_uri = re.sub('spotify:track:', '', mpd_slice['playlists'][idx]['tracks'][idy]['track_uri'])
        if(track_uri not in track_dict):
            track_dict[track_uri] = [count, mpd_slice['playlists'][idx]['tracks'][idy]['track_uri']]
            count += 1
        lst_current.append(track_dict[track_uri][0])
    X_train.append(lst_current)

print (X_train[0:100])
print (count)
print (len(play_list_names))
vocab = []
for x in X_train:
    vocab.extend(x)
    
print (vocab)


[[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147], [2, 148, 149, 150, 151, 152, 153, 4, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210], [2, 211, 212, 213, 214, 215, 216, 217, 218, 219, 2

In [5]:


def maybe_download(filename, url, expected_bytes):
    """Download a file if not present, and make sure it's the right size."""
    if not os.path.exists(filename):
        filename, _ = urllib.request.urlretrieve(url + filename, filename)
    statinfo = os.stat(filename)
    if statinfo.st_size == expected_bytes:
        print('Found and verified', filename)
    else:
        print(statinfo.st_size)
        raise Exception(
            'Failed to verify ' + filename + '. Can you get to it with a browser?')
    return filename

In [6]:
# Read the data into a list of strings.
def read_data(filename):
    """Extract the first file enclosed in a zip file as a list of words."""
    with zipfile.ZipFile(filename) as f:
        data = tf.compat.as_str(f.read(f.namelist()[0])).split()
    return data

In [7]:
def build_dataset(words, n_words):
    """Process raw inputs into a dataset."""
    count = [['UNK', -1]]
    count.extend(collections.Counter(words).most_common(n_words - 1))
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0
    for word in words:
        if word in dictionary:
            index = dictionary[word]
        else:
            index = 0  # dictionary['UNK']
            unk_count += 1
        data.append(index)
    count[0][1] = unk_count
    reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reversed_dictionary

In [8]:
def collect_data(vocabulary_size=10000):
    url = 'http://mattmahoney.net/dc/'
    filename = maybe_download('text8.zip', url, 31344016)
    #vocabulary = read_data(filename)
    vocabulary = vocab
    #print(vocabulary[:100])
    data, count, dictionary, reverse_dictionary = build_dataset(vocabulary,
                                                                vocabulary_size)
    del vocabulary  # Hint to reduce memory.
    return data, count, dictionary, reverse_dictionary

In [9]:
vocab_size = count
data, count, dictionary, reverse_dictionary = collect_data(vocabulary_size=vocab_size)
print(data[:7])

window_size = 20
vector_dim = 300
epochs = 200000

valid_size = 16     # Random set of words to evaluate similarity on.
valid_window = 100  # Only pick dev samples in the head of the distribution.
valid_examples = np.random.choice(valid_window, valid_size, replace=False)

('Found and verified', 'text8.zip')
[1, 1281, 101, 1739, 6461, 1946, 343]


In [10]:
sampling_table = sequence.make_sampling_table(vocab_size)
couples, labels = skipgrams(data, vocab_size, window_size=window_size, sampling_table=sampling_table)
word_target, word_context = zip(*couples)
word_target = np.array(word_target, dtype="int32")
word_context = np.array(word_context, dtype="int32")

print("Printing coiples", couples[:10], labels[:10])

('Printing coiples', [[22608, 1512], [3214, 3196], [38271, 95], [42040, 25473], [42051, 29381], [27042, 39212], [2233, 50720], [1600, 15009], [1360, 18425], [1826, 50727]], [1, 0, 1, 0, 0, 0, 0, 0, 1, 1])


In [11]:
# create some input variables
input_target = Input((1,))
input_context = Input((1,))

embedding = Embedding(vocab_size, vector_dim, input_length=1, name='embedding')
target = embedding(input_target)
target = Reshape((vector_dim, 1))(target)
context = embedding(input_context)
context = Reshape((vector_dim, 1))(context)

# setup a cosine similarity operation which will be output in a secondary model
similarity = merge([target, context], mode='cos', dot_axes=0)

# now perform the dot product operation to get a similarity measure
dot_product = merge([target, context], mode='dot', dot_axes=1)
dot_product = Reshape((1,))(dot_product)
# add the sigmoid output layer
output = Dense(1, activation='sigmoid')(dot_product)
# create the primary training model
model = Model(input=[input_target, input_context], output=output)
model.compile(loss='binary_crossentropy', optimizer='rmsprop')

# create a secondary validation model to run our similarity checks during training
validation_model = Model(input=[input_target, input_context], output=similarity)

/home/rgsl888/ana/envs/tf27/lib/python2.7/site-packages/ipykernel/__main__.py:12: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/home/rgsl888/ana/envs/tf27/lib/python2.7/site-packages/keras/legacy/layers.py:465: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)
/home/rgsl888/ana/envs/tf27/lib/python2.7/site-packages/ipykernel/__main__.py:15: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/home/rgsl888/ana/envs/tf27/lib/python2.7/site-packages/ipykernel/__main__.py:20: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=[<tf.Tenso...)`
/home/rgsl888/ana/envs/tf27/lib/python2.7/site-packages/ipykernel/

In [12]:
class SimilarityCallback:
    def run_sim(self):
        for i in range(valid_size):
            valid_word = reverse_dictionary[valid_examples[i]]
            top_k = 8  # number of nearest neighbors
            sim = self._get_sim(valid_examples[i])
            nearest = (-sim).argsort()[1:top_k + 1]
            log_str = 'Nearest to %s:' % valid_word
            for k in range(top_k):
                close_word = reverse_dictionary[nearest[k]]
                log_str = '%s %s,' % (log_str, close_word)
            print(log_str)

    @staticmethod
    def _get_sim(valid_word_idx):
        sim = np.zeros((vocab_size,))
        in_arr1 = np.zeros((1,))
        in_arr2 = np.zeros((1,))
        in_arr1[0,] = valid_word_idx
        for i in range(vocab_size):
            in_arr2[0,] = i
            out = validation_model.predict_on_batch([in_arr1, in_arr2])
            sim[i] = out
        return sim
sim_cb = SimilarityCallback()

In [13]:
arr_1 = np.zeros((1,))
arr_2 = np.zeros((1,))
arr_3 = np.zeros((1,))
for cnt in range(epochs):
    idx = np.random.randint(0, len(labels)-1)
    arr_1[0,] = word_target[idx]
    arr_2[0,] = word_context[idx]
    arr_3[0,] = labels[idx]
    loss = model.train_on_batch([arr_1, arr_2], arr_3)
    if cnt % 100 == 0:
        print("Iteration {}, loss={}".format(cnt, loss))
    if cnt % 40000 == 0:
        sim_cb.run_sim()

Iteration 0, loss=0.690884292126
Nearest to 3389: 1495, 14634, 54748, 47845, 46066, 8244, 12093, 10752,
Nearest to 1887: 25911, 38964, 20360, 15977, 56697, 40661, 25579, 35761,
Nearest to 1281: 25546, 56441, 39997, 13105, 47853, 55766, 27271, 14497,
Nearest to 2246: 21012, 12659, 18046, 18793, 2447, 23060, 59047, 6527,
Nearest to 80: 44924, 24529, 9715, 2280, 83, 56382, 53592, 23453,
Nearest to 4205: 51521, 2797, 4991, 55919, 41345, 14410, 29578, 46105,
Nearest to 65: 12654, 55245, 54795, 46119, 14785, 29549, 30687, 30203,
Nearest to 2435: 38403, 29430, 40367, 41614, 11508, 54670, 24641, 56943,
Nearest to 629: 22978, 47376, 46114, 3212, 52638, 56485, 56311, 55137,
Nearest to 16: 13082, 48509, 49703, 54537, 48559, 5836, 5539, 51451,
Nearest to 250: 4322, 56696, 53421, 50669, 32622, 43896, 44228, 49636,
Nearest to 218: 2592, 37457, 55613, 33395, 5129, 51355, 24132, 41161,
Nearest to 13: 8635, 37670, 43756, 27718, 57748, 43882, 20643, 57254,
Nearest to 3698: 16177, 970, 31552, 23537, 5137

Iteration 19500, loss=0.737470090389
Iteration 19600, loss=0.733040809631
Iteration 19700, loss=0.733119904995
Iteration 19800, loss=0.74156922102
Iteration 19900, loss=0.737941622734
Iteration 20000, loss=0.648308753967
Iteration 20100, loss=0.725880920887
Iteration 20200, loss=0.713755905628
Iteration 20300, loss=0.668769359589
Iteration 20400, loss=0.727384567261
Iteration 20500, loss=0.661513447762
Iteration 20600, loss=0.735221326351
Iteration 20700, loss=0.645548820496
Iteration 20800, loss=0.650361359119
Iteration 20900, loss=0.649460732937
Iteration 21000, loss=0.650778830051
Iteration 21100, loss=0.650633394718
Iteration 21200, loss=0.733158826828
Iteration 21300, loss=0.650458276272
Iteration 21400, loss=0.649503707886
Iteration 21500, loss=0.736666381359
Iteration 21600, loss=0.658733010292
Iteration 21700, loss=0.657342493534
Iteration 21800, loss=0.733831048012
Iteration 21900, loss=0.65426248312
Iteration 22000, loss=0.663530051708
Iteration 22100, loss=0.731654047966
Ite

Nearest to 16: 17334, 12069, 12462, 2742, 41944, 47038, 20247, 10017,
Nearest to 250: 16702, 4322, 55265, 42044, 15776, 56696, 13390, 44228,
Nearest to 218: 37457, 41161, 55613, 33395, 57255, 39363, 35074, 39537,
Nearest to 13: 8635, 14431, 13897, 13257, 44090, 43756, 8896, 43882,
Nearest to 3698: 16177, 970, 32437, 31552, 588, 58422, 57421, 55929,
Nearest to 533: 14540, 15685, 8860, 1344, 24669, 5917, 45187, 50268,
Nearest to 379: 3001, 14834, 18194, 11887, 39799, 1276, 6080, 31411,
Iteration 40100, loss=0.712137401104
Iteration 40200, loss=0.677697002888
Iteration 40300, loss=0.683077871799
Iteration 40400, loss=0.687299251556
Iteration 40500, loss=0.69690310955
Iteration 40600, loss=0.70589184761
Iteration 40700, loss=0.70949190855
Iteration 40800, loss=0.670414209366
Iteration 40900, loss=0.671756625175
Iteration 41000, loss=0.677385032177
Iteration 41100, loss=0.670939207077
Iteration 41200, loss=0.707393586636
Iteration 41300, loss=0.672406435013
Iteration 41400, loss=0.709792613

Iteration 61100, loss=0.735805392265
Iteration 61200, loss=0.648196280003
Iteration 61300, loss=0.632159948349
Iteration 61400, loss=0.751203238964
Iteration 61500, loss=0.638266324997
Iteration 61600, loss=0.645488262177
Iteration 61700, loss=0.751210927963
Iteration 61800, loss=0.650430738926
Iteration 61900, loss=0.737720370293
Iteration 62000, loss=0.722323417664
Iteration 62100, loss=0.667312800884
Iteration 62200, loss=0.717530548573
Iteration 62300, loss=0.669491708279
Iteration 62400, loss=0.65361315012
Iteration 62500, loss=0.71533113718
Iteration 62600, loss=0.67360240221
Iteration 62700, loss=0.670926749706
Iteration 62800, loss=0.727646231651
Iteration 62900, loss=0.675443768501
Iteration 63000, loss=0.704939961433
Iteration 63100, loss=0.694426000118
Iteration 63200, loss=0.688859879971
Iteration 63300, loss=0.704272687435
Iteration 63400, loss=0.709683835506
Iteration 63500, loss=0.677879750729
Iteration 63600, loss=0.696500658989
Iteration 63700, loss=0.661683082581
Iter

Iteration 80300, loss=0.710948228836
Iteration 80400, loss=0.721636533737
Iteration 80500, loss=0.663977384567
Iteration 80600, loss=0.664623022079
Iteration 80700, loss=0.720840513706
Iteration 80800, loss=0.723529934883
Iteration 80900, loss=0.664689898491
Iteration 81000, loss=0.662585556507
Iteration 81100, loss=0.717596650124
Iteration 81200, loss=0.674767196178
Iteration 81300, loss=0.70560002327
Iteration 81400, loss=0.723444521427
Iteration 81500, loss=0.681289434433
Iteration 81600, loss=0.701232016087
Iteration 81700, loss=0.697877705097
Iteration 81800, loss=0.662765145302
Iteration 81900, loss=0.696587562561
Iteration 82000, loss=0.679980576038
Iteration 82100, loss=0.697366595268
Iteration 82200, loss=0.698353528976
Iteration 82300, loss=0.691254019737
Iteration 82400, loss=0.694326400757
Iteration 82500, loss=0.689835548401
Iteration 82600, loss=0.679246068001
Iteration 82700, loss=0.687056064606
Iteration 82800, loss=0.699572086334
Iteration 82900, loss=0.702048957348
It

Iteration 102500, loss=0.664811134338
Iteration 102600, loss=0.68570625782
Iteration 102700, loss=0.710812628269
Iteration 102800, loss=0.701848983765
Iteration 102900, loss=0.712512016296
Iteration 103000, loss=0.726459622383
Iteration 103100, loss=0.677967131138
Iteration 103200, loss=0.732933044434
Iteration 103300, loss=0.720041930676
Iteration 103400, loss=0.705027878284
Iteration 103500, loss=0.648888230324
Iteration 103600, loss=0.720995545387
Iteration 103700, loss=0.732739806175
Iteration 103800, loss=0.669922888279
Iteration 103900, loss=0.676520586014
Iteration 104000, loss=0.712952911854
Iteration 104100, loss=0.680001080036
Iteration 104200, loss=0.711409211159
Iteration 104300, loss=0.673603236675
Iteration 104400, loss=0.696627914906
Iteration 104500, loss=0.699623525143
Iteration 104600, loss=0.709080696106
Iteration 104700, loss=0.683170497417
Iteration 104800, loss=0.695011973381
Iteration 104900, loss=0.703727841377
Iteration 105000, loss=0.687399923801
Iteration 105

Iteration 121200, loss=0.702923715115
Iteration 121300, loss=0.692964792252
Iteration 121400, loss=0.697167038918
Iteration 121500, loss=0.683780312538
Iteration 121600, loss=0.714533507824
Iteration 121700, loss=0.681437671185
Iteration 121800, loss=0.687630534172
Iteration 121900, loss=0.675035297871
Iteration 122000, loss=0.698252022266
Iteration 122100, loss=0.664532423019
Iteration 122200, loss=0.66329485178
Iteration 122300, loss=0.694593548775
Iteration 122400, loss=0.686910092831
Iteration 122500, loss=0.685737907887
Iteration 122600, loss=0.64178198576
Iteration 122700, loss=0.721972942352
Iteration 122800, loss=0.721053600311
Iteration 122900, loss=0.680901646614
Iteration 123000, loss=0.657957017422
Iteration 123100, loss=0.685156345367
Iteration 123200, loss=0.708329200745
Iteration 123300, loss=0.672277927399
Iteration 123400, loss=0.704670548439
Iteration 123500, loss=0.687958836555
Iteration 123600, loss=0.69315212965
Iteration 123700, loss=0.68868714571
Iteration 123800

Iteration 142900, loss=0.704788386822
Iteration 143000, loss=0.726185619831
Iteration 143100, loss=0.690017879009
Iteration 143200, loss=0.710913598537
Iteration 143300, loss=0.752301394939
Iteration 143400, loss=0.699626922607
Iteration 143500, loss=0.702784657478
Iteration 143600, loss=0.683576047421
Iteration 143700, loss=0.678877592087
Iteration 143800, loss=0.710281133652
Iteration 143900, loss=0.711086452007
Iteration 144000, loss=0.655705749989
Iteration 144100, loss=0.683007657528
Iteration 144200, loss=0.737981438637
Iteration 144300, loss=0.740414500237
Iteration 144400, loss=0.688853681087
Iteration 144500, loss=0.694991588593
Iteration 144600, loss=0.680301904678
Iteration 144700, loss=0.662710785866
Iteration 144800, loss=0.66668432951
Iteration 144900, loss=0.66208422184
Iteration 145000, loss=0.726562023163
Iteration 145100, loss=0.706392347813
Iteration 145200, loss=0.643076658249
Iteration 145300, loss=0.704575300217
Iteration 145400, loss=0.709147036076
Iteration 1455

Iteration 161600, loss=0.724794447422
Iteration 161700, loss=0.726100564003
Iteration 161800, loss=0.646600961685
Iteration 161900, loss=0.710424304008
Iteration 162000, loss=0.655288219452
Iteration 162100, loss=0.641487896442
Iteration 162200, loss=0.639993548393
Iteration 162300, loss=0.744605362415
Iteration 162400, loss=0.656277000904
Iteration 162500, loss=0.699275970459
Iteration 162600, loss=0.633540928364
Iteration 162700, loss=0.72726392746
Iteration 162800, loss=0.70484572649
Iteration 162900, loss=0.695738971233
Iteration 163000, loss=0.744589805603
Iteration 163100, loss=0.676079332829
Iteration 163200, loss=0.645284116268
Iteration 163300, loss=0.791892766953
Iteration 163400, loss=0.737880468369
Iteration 163500, loss=0.730481386185
Iteration 163600, loss=0.710333585739
Iteration 163700, loss=0.746339023113
Iteration 163800, loss=0.751842021942
Iteration 163900, loss=0.719491004944
Iteration 164000, loss=0.752125024796
Iteration 164100, loss=0.644807934761
Iteration 1642

Iteration 183300, loss=0.695466935635
Iteration 183400, loss=0.686710834503
Iteration 183500, loss=0.646371841431
Iteration 183600, loss=0.677564918995
Iteration 183700, loss=0.642398476601
Iteration 183800, loss=0.642972111702
Iteration 183900, loss=0.664790332317
Iteration 184000, loss=0.682727694511
Iteration 184100, loss=0.723203063011
Iteration 184200, loss=0.683465242386
Iteration 184300, loss=0.746734559536
Iteration 184400, loss=0.68541020155
Iteration 184500, loss=0.673823952675
Iteration 184600, loss=0.776138246059
Iteration 184700, loss=0.726611852646
Iteration 184800, loss=0.719168186188
Iteration 184900, loss=0.694269657135
Iteration 185000, loss=0.660989999771
Iteration 185100, loss=0.633253693581
Iteration 185200, loss=0.713536262512
Iteration 185300, loss=0.679021954536
Iteration 185400, loss=0.67978399992
Iteration 185500, loss=0.676680743694
Iteration 185600, loss=0.721341907978
Iteration 185700, loss=0.709226429462
Iteration 185800, loss=0.676389098167
Iteration 1859

In [14]:
#print("Printing coiples", couples[:10], labels[:10])

In [14]:
if 'session' in locals() and session is not None:
    print('Close interactive session')
    session.close()


In [15]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 1, 300)       17758500    input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
reshape_1 (Reshape)             (None, 300, 1)       0           embedding[0][0]                  
__________

In [16]:
wt = embedding.get_weights()
s = wt[0]
print (s.shape)

(59195, 300)


In [17]:
np.save("embedding.npy", s)